<a href="https://colab.research.google.com/github/changdaeoh/DACON_Dirty-MNIST/blob/main/0218_fit_on_newdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글드라이브와 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# setting
import tensorflow as tf
from tensorflow import  keras

!pip install tensorflow-addons
import tensorflow_addons as tfa

import numpy as np
import pandas as pd 

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
os.chdir('/content/drive/MyDrive/project_dataset/dacon_v2')


# 데이터 경로
train_dir = "/content/drive/MyDrive/project_dataset/dacon_v2/dirty_mnist_2nd"
test_dir = "/content/drive/MyDrive/project_dataset/dacon_v2/test_route"

# GPU 확인
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# # 현재 경로에 새로운 디렉토리 생성
# # 생성한 디렉토리에 zip파일 압축해제

from google.colab import output

!mkdir "dirty_mnist_2nd"
!unzip "dirty_mnist_2nd.zip" -d "dirty_mnist_2nd"

# !mkdir "test_dirty_mnist"
# !unzip "test_dirty_mnist.zip" -d "./test_dirty_mnist"

output.clear()

# Training 
### 1. Data Preparing

In [ ]:
meta_df = pd.read_csv('dirty_mnist_2nd_answer.csv')
meta_df['index'] = meta_df['index'].apply(lambda x: str("{0:05d}".format(x))+'.png')
columns = list(meta_df.columns[1:])


# Data Augumentation
datagen_ag = ImageDataGenerator(rescale=1./255., 
                               rotation_range = 10,
                               width_shift_range = 0.2,
                               height_shift_range = 0.2,
                               horizontal_flip = True,
                               vertical_flip = True,
                               validation_split=0.1,
                               fill_mode = "nearest")

# generator_ag
train_gen_ag = datagen_ag.flow_from_dataframe(dataframe = meta_df,        
                                        directory = train_dir,       
                                        x_col='index',               
                                        y_col=columns,                
                                        batch_size = 32,               
                                        seed = 42,
                                        color_mode = "rgb",           
                                        class_mode='raw',
                                        target_size=(256, 256),       
                                        subset='training')
val_gen_ag = datagen_ag.flow_from_dataframe(dataframe = meta_df,
                                        directory = train_dir,
                                        x_col='index',
                                        y_col=columns,
                                        batch_size = 32,
                                        seed = 42,
                                        color_mode = "rgb",
                                        class_mode='raw',
                                        target_size=(256, 256),
                                        subset='validation')


# datagen_1 = ImageDataGenerator(rescale=1./255., validation_split=0.1)

# # not ag
# train_gen = datagen_1.flow_from_dataframe(dataframe = meta_df,        
#                                         directory = train_dir,       
#                                         x_col='index',               
#                                         y_col=columns,                
#                                         batch_size = 32,               
#                                         seed = 42,
#                                         color_mode = "rgb",           
#                                         class_mode='raw',
#                                         target_size=(256, 256),       
#                                         subset='training')
# val_gen = datagen_1.flow_from_dataframe(dataframe = meta_df,
#                                         directory = train_dir,
#                                         x_col='index',
#                                         y_col=columns,
#                                         batch_size = 32,
#                                         seed = 42,
#                                         color_mode = "rgb",
#                                         class_mode='raw',
#                                         target_size=(256, 256),
#                                         subset='validation')

Found 45000 validated image filenames.
Found 5000 validated image filenames.


### 2. Model Building

In [ ]:
base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False,
                                                                      weights='imagenet',
                                                                      input_shape = (256, 256, 3))
# model body
incep_res = keras.Sequential([
                              base_model,

                              keras.layers.GlobalAveragePooling2D(), 
                              keras.layers.Dense(1024, kernel_initializer='he_normal'),
                              keras.layers.BatchNormalization(),
                              keras.layers.Activation('elu'),
                              keras.layers.Dense(512, kernel_initializer='he_normal'),
                              keras.layers.BatchNormalization(),
                              keras.layers.Activation('elu'),
                              keras.layers.Dense(256, kernel_initializer='he_normal'),
                              keras.layers.BatchNormalization(),
                              keras.layers.Activation('elu'),
                              keras.layers.Dense(26, kernel_initializer='glorot_normal', activation='sigmoid')
])      

# optimizer & metrics
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,
#                                                           decay_steps=10000,
#                                                           decay_rate=0.9)
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
BAcc = keras.metrics.BinaryAccuracy()

# model compile
incep_res.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = [BAcc])

# callbacks
checkpoint = keras.callbacks.ModelCheckpoint(f'./model/incep_res_cd_0219_v1.h5', 
                                             save_best_only=True, verbose=1)
early_stop_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

# fitting
history = incep_res.fit(train_gen_ag, epochs = 35, 
                        validation_data = val_gen_ag, 
                        callbacks = [checkpoint, early_stop_cb])

219062272/219055592 [==============================] - 5s 0us/step
Epoch 1/35
1407/1407 [==============================] - 895s 605ms/step - loss: 0.6916 - binary_accuracy: 0.5485 - val_loss: 0.6706 - val_binary_accuracy: 0.5936

Epoch 00001: val_loss improved from inf to 0.67056, saving model to ./model/incep_res_cd_0219_v1.h5
Epoch 2/35
1407/1407 [==============================] - 850s 604ms/step - loss: 0.6416 - binary_accuracy: 0.6228 - val_loss: 0.6288 - val_binary_accuracy: 0.6420

Epoch 00002: val_loss improved from 0.67056 to 0.62884, saving model to ./model/incep_res_cd_0219_v1.h5
Epoch 3/35
1407/1407 [==============================] - 852s 605ms/step - loss: 0.6100 - binary_accuracy: 0.6574 - val_loss: 0.6182 - val_binary_accuracy: 0.6619

Epoch 00003: val_loss improved from 0.62884 to 0.61823, saving model to ./model/incep_res_cd_0219_v1.h5
Epoch 4/35
1407/1407 [==============================] - 854s 606ms/step - loss: 0.5875 - binary_accuracy: 0.6795 - val_loss: 0.5993 - va

### 훈련데이터를 더 써서 두 번째 훈련
* 49000 : 1000

In [ ]:
# Data Augumentation
datagen_ag2 = ImageDataGenerator(rescale=1./255., 
                               rotation_range = 10,
                               width_shift_range = 0.2,
                               height_shift_range = 0.2,
                               horizontal_flip = True,
                               vertical_flip = True,
                               validation_split=0.02,
                               fill_mode = "nearest")

# generator_ag
train_gen_ag2 = datagen_ag2.flow_from_dataframe(dataframe = meta_df,        
                                        directory = train_dir,       
                                        x_col='index',               
                                        y_col=columns,                
                                        batch_size = 32,               
                                        seed = 42,
                                        color_mode = "rgb",           
                                        class_mode='raw',
                                        target_size=(256, 256),       
                                        subset='training')
val_gen_ag2 = datagen_ag2.flow_from_dataframe(dataframe = meta_df,
                                        directory = train_dir,
                                        x_col='index',
                                        y_col=columns,
                                        batch_size = 32,
                                        seed = 42,
                                        color_mode = "rgb",
                                        class_mode='raw',
                                        target_size=(256, 256),
                                        subset='validation')

Found 49000 validated image filenames.
Found 1000 validated image filenames.


In [ ]:
# callbacks
checkpoint2 = keras.callbacks.ModelCheckpoint(f'./model/incep_res_cd_0219_v1_add.h5', 
                                             save_best_only=True, verbose=1)
early_stop_cb2 = keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True)

# fitting
history = incep_res.fit(train_gen_ag2, epochs = 10, 
                        validation_data = val_gen_ag2, 
                        callbacks = [checkpoint2, early_stop_cb2])

Epoch 1/10
1532/1532 [==============================] - 811s 529ms/step - loss: 0.2813 - binary_accuracy: 0.8869 - val_loss: 0.3239 - val_binary_accuracy: 0.8685

Epoch 00001: val_loss improved from inf to 0.32388, saving model to ./model/incep_res_cd_0219_v1_add.h5
Epoch 2/10
1532/1532 [==============================] - 822s 536ms/step - loss: 0.2768 - binary_accuracy: 0.8894 - val_loss: 0.3171 - val_binary_accuracy: 0.8703

Epoch 00002: val_loss improved from 0.32388 to 0.31705, saving model to ./model/incep_res_cd_0219_v1_add.h5
Epoch 3/10
1532/1532 [==============================] - 826s 539ms/step - loss: 0.2731 - binary_accuracy: 0.8909 - val_loss: 0.3142 - val_binary_accuracy: 0.8726

Epoch 00003: val_loss improved from 0.31705 to 0.31417, saving model to ./model/incep_res_cd_0219_v1_add.h5
Epoch 4/10
1532/1532 [==============================] - 826s 539ms/step - loss: 0.2701 - binary_accuracy: 0.8921 - val_loss: 0.3211 - val_binary_accuracy: 0.8670

Epoch 00004: val_loss did no

KeyboardInterrupt: ignored

# Prediction

In [ ]:
# image_dataset_from_directory
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = test_dir,
    label_mode = None,
    class_names = None,
    color_mode = "rgb",
    batch_size = 64,
    image_size = (256, 256),
    shuffle = False,
    seed = None,
    validation_split = None
)

test_ds = test_ds.prefetch(1)

Found 5000 files belonging to 1 classes.


In [ ]:
# add rescaling layer
mod = keras.Sequential([
                  keras.layers.experimental.preprocessing.Rescaling(1./255.),
                  incep_res
])

# prediction
result = mod.predict(test_ds)
result2 = result.copy()

# get class label
result2[result2 <= 0.5] = 0
result2[result2 > 0.5] = 1

# frame create
res = pd.DataFrame(result2, columns = columns )
int_rest = res.astype(int)
submit = pd.concat([test_df.iloc[:,0],int_rest], axis = 1)


submit.to_csv("IncepRes_0219_v1.csv", index = False)


In [ ]:
# 모든 데이터 써서 최종 훈련

# Data Augumentation
datagen_fin = ImageDataGenerator(rescale=1./255.)

# generator_ag
train_gen_fin = datagen_fin.flow_from_dataframe(dataframe = meta_df,        
                                        directory = train_dir,       
                                        x_col='index',               
                                        y_col=columns,                
                                        batch_size = 32,               
                                        seed = 42,
                                        color_mode = "rgb",           
                                        class_mode='raw',
                                        target_size=(256, 256),       
                                        subset='training')


# opt
optimizer = keras.optimizers.Nadam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
BAcc = keras.metrics.BinaryAccuracy()

# model compile
incep_res.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = [BAcc])


# callbacks
checkpoint3 = keras.callbacks.ModelCheckpoint(f'./model/incep_res_cd_0219_v1_fin.h5', verbose=1)

# fitting
history_fin = incep_res.fit(train_gen_fin, epochs = 2, callbacks = [checkpoint3])

Found 50000 validated image filenames.
Epoch 1/2
1563/1563 [==============================] - 522s 306ms/step - loss: 0.1179 - binary_accuracy: 0.9588
Epoch 2/2
1563/1563 [==============================] - 478s 306ms/step - loss: 0.0701 - binary_accuracy: 0.9764


In [ ]:
incep_res.save('./model/incep_res_cd_0219_v1_fin.h5')

In [ ]:
mod = keras.Sequential([
                  keras.layers.experimental.preprocessing.Rescaling(1./255.),
                  incep_res
])

# make a prediction on the test set
yhat = mod.predict(test_ds)
# round probabilities to class labels
yhat = yhat.round()


# frame create
res = pd.DataFrame(yhat, columns = columns )
int_rest = res.astype(int)
submit = pd.concat([test_df.iloc[:,0],int_rest], axis = 1)

submit.to_csv("IncepRes_0219_v2.csv", index = False)